In [4]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import os
import gc

import keras as k
from keras import optimizers
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import Convolution2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.cross_validation import KFold
from sklearn.metrics import fbeta_score, precision_score, recall_score

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns

import cv2
from tqdm import tqdm

from datetime import datetime
import time
import configparser
import json
import sys

from utils.file import makedirs
from utils.recorder import record_model_medata, record_model_scores
from utils.loader import *
from utils.f2thresholdfinder import *
from utils.imagegen import *
from utils.models import *
from utils.custommetrics import *
from utils.samplesduplicator import duplicate_train_samples
from utils.training import *
from utils.predictor import *
from utils.augmentation import *
from utils.generator import *


In [5]:
timestr = time.strftime("%Y%m%d-%H%M%S")
start_time = datetime.now()

In [6]:
config_file = 'cfg/default.cfg'
#config_file = 'cfg/11_rgb.cfg'

# command line args processing "python aggregate_model.py cfg/3.cfg"
if len(sys.argv) > 1 and '.cfg' in sys.argv[1]:
    config_file = sys.argv[1]

print('reading configurations from config file: {}'.format(config_file))

settings = configparser.ConfigParser()
settings.read(config_file)
data_dir = settings.get('data', 'data_dir')

df_train = pd.read_csv(data_dir + 'train_v2.csv')
model_filename = 'aggregate_model_'+ timestr +'.h5'
model_filepath = data_dir + 'models/' + model_filename
sample_submission_filepath = data_dir + 'sample_submission_v2.csv'
number_of_samples = len(df_train.index)
print('total number of samples: {}'.format(number_of_samples))

# WARNING: keras allow either 1, 3, or 4 channels per pixel. Other numbers not allowed.
data_mask_label = np.array(['R', 'G', 'B', 'NDVI', 'NDWI', 'NIR'])
#print(settings.get('data', 'data_mask'))
data_mask_list = json.loads(settings.get('data', 'data_mask'))

data_mask = ma.make_mask(data_mask_list)
print(data_mask)

num_channels = np.sum(data_mask)
need_norm_stats = False

model_id = settings.get('model', 'model_id')
print('model: {}'.format(model_id))

learning_rate_schedule = json.loads(settings.get('model', 'learning_rate_schedule'))
max_epoch_per_learning_rate = json.loads(settings.get('model', 'max_epoch_per_learning_rate'))
print('learning rates: {}'.format(learning_rate_schedule))
print('with max epoch: {}'.format(max_epoch_per_learning_rate))

# default to 64
rescaled_dim = 64
if settings.has_option('data', 'rescaled_dim'):
    rescaled_dim = settings.getint('data', 'rescaled_dim')
print('rescaled dimension: {}'.format(rescaled_dim))

# one epoch is an arbitrary cutoff : one pass over the entire training set

# a batch results in exactly one update to the model.
# batch_size is limited by model size and GPU memory
batch_size = settings.getint('model', 'batch_size') 
print('batch size: {}'.format(batch_size))

classifier_threshold = 0.2 # used for end of epoch f2 approximation only

split = int(number_of_samples * 0.80)  # TODO we may want to increase to 0.90 eventually
number_validations = number_of_samples - split

has_augmentation_config = settings.has_section('augmentation')
if has_augmentation_config:
    rotation_range = settings.getint('augmentation', 'rotation_range')
    horizontal_flip = settings.getboolean('augmentation', 'horizontal_flip')
    vertical_flip = settings.getboolean('augmentation', 'vertical_flip')
    print('rotation_range:{} horizontal_flip:{} vertical_flip:{}'.format(rotation_range,horizontal_flip,vertical_flip))


reading configurations from config file: cfg/default.cfg


C:\Users\Me\Anaconda2\lib\site-packages\ipykernel\__main__.py:11: DeprecationWarning: You passed a bytestring as `filenames`. This will not work on Python 3. Use `cp.read_file()` or switch to using Unicode strings across the board.


total number of samples: 40479
[ True  True  True False False False]
model: JAGG_2
learning rates: [0.001, 0.0001, 1e-05]
with max epoch: [3, 1, 1]
rescaled dimension: 64
batch size: 512
rotation_range:20 horizontal_flip:True vertical_flip:True


In [7]:
flatten = lambda l: [item for sublist in l for item in sublist]
labels = list(set(flatten([l.split(' ') for l in df_train['tags'].values])))

print(labels)
print(len(labels))

['slash_burn', 'clear', 'blooming', 'primary', 'cloudy', 'conventional_mine', 'water', 'haze', 'cultivation', 'partly_cloudy', 'artisinal_mine', 'habitation', 'bare_ground', 'blow_down', 'agriculture', 'road', 'selective_logging']
17


In [8]:
x_train, y_train = load_training_set(df_train, rescaled_dim)
print(x_train.shape)
print(y_train.shape)

(40479L, 64L, 64L, 6L)
(40479L, 17L)


In [9]:
x_train = x_train[:, :, :, data_mask]

x_train = x_train.transpose(0,3,1,2)  # https://github.com/fchollet/keras/issues/2681
print(x_train.shape)

(40479L, 3L, 64L, 64L)


In [10]:
# TODO save the shuffling order to hdf5 so we can recreate the training and validation sets post execution.

# shuffle the samples because 
# 1) the original samples may not be randomized & 
# 2) to avoid the possiblility of overfitting the validation data while we tune the model
from sklearn.utils import shuffle
x_train, y_train = shuffle(x_train, y_train, random_state=0)

x_train, x_valid, y_train, y_valid = x_train[:split], x_train[split:], y_train[:split], y_train[split:]

In [11]:
print(x_train.shape)
print(y_train.shape)
print(x_valid.shape)
print(y_valid.shape)

(32383L, 3L, 64L, 64L)
(32383L, 17L)
(8096L, 3L, 64L, 64L)
(8096L, 17L)


In [12]:
# experimental hack to get more samples for augmentations for a specific low-frequency tag in unbalanced dataset. e.g. habitation
# selecting the optimal multiplier is sensitive

# augmentation_hack_config = settings.has_section('augmentation_hack')
# if augmentation_hack_config:
#     dup_multiplier = settings.getint('augmentation_hack', 'multiplier')
#     hack_label_target = settings.get('augmentation_hack', 'label_target')

# if augmentation_hack_config:
#     x_train, y_train = duplicate_train_samples(x_train, y_train, labels.index(hack_label_target), multiplier=dup_multiplier)
#     print(x_train.shape)
#     print(y_train.shape)


In [13]:
# dynamicly set num_samples_per_epoch
# TODO understand the implications of num_samples_per_epoch.
# +0.002 to +0.01??? F2 score improvement when number_of_samples * 3
# num_samples_per_epoch = 1000
num_samples_per_epoch = x_train.shape[0]

In [14]:
single_taget_model = False
# warning: experimental. 
# shuffling won't let you put things back together
if settings.has_option('data', 'single_target'):
    single_taget_model = True
    single_target_label = settings.get('data', 'single_target')
    single_target_label_index = labels.index(single_target_label)
    y_train = y_train[:,single_target_label_index]
    y_valid = y_valid[:,single_target_label_index]
    
score_averaging_method = 'binary' if single_taget_model else 'samples'
print('score_averaging_method', score_averaging_method)

('score_averaging_method', 'samples')


In [15]:
def get_img_generator():
    if has_augmentation_config:
        return GeneralImgGen(rotation_range = rotation_range, 
                             horizontal_flip = horizontal_flip, 
                             vertical_flip = vertical_flip)
    else:
        return ScaledDown() # default
    
image_generator = get_img_generator()
print('image generator', image_generator)

('image generator', rotation_range:20 horizontal_flip:True vertical_flip:True)


In [16]:
# this is the augmentation configuration we will use for training
train_datagen = image_generator.getTrainGenenerator()

In [17]:
if (need_norm_stats):
    # need to compute internal stats like featurewise std and zca whitening
    train_datagen.fit(x_train)

In [18]:
validation_datagen = image_generator.getValidationGenenerator()

In [19]:
# workaround to provide your own stats: 
# http://stackoverflow.com/questions/41855512/how-does-data-normalization-work-in-keras-during-prediction/43069409#43069409
if (need_norm_stats):
    # need to compute internal stats like featurewise std and zca whitening
    validation_datagen.fit(x_valid)

In [20]:
# validation_generator = validation_datagen.flow(
#         x_valid,
#         y_valid,
#         batch_size=batch_size,
#         shuffle=False)

In [21]:
if single_taget_model:
    set_model_output_layer_size(1)
    
model = get_model(model_id, num_channels, rescaled_dim, rescaled_dim)


In [22]:
# BUG when resuming training, the learning rate need to be decreased.
# let's load an existing trained model and continue training more epoch gives 0.01 improvement in LB score.
# model = load_model(data_dir + 'models/aggregate_model_20170507-124128.h5') # 0.86
# model = load_model(data_dir + 'models/aggregate_model_20170507-184232.h5') # 0.87
# model = load_model(data_dir + 'models/aggregate_model_20170511-133235.h5')
# model = load_model(data_dir + 'models/aggregate_model_20170515-062741.h5')

In [35]:
# Note: threshold is fixed (not optimized per label)
def compute_f2_measure(l_model, x_data, y_data):    
    val_generator_f2 = validation_datagen.flow(
        x_data,
        y_data,
        batch_size=64,
        shuffle=False)
    raw_pred = l_model.predict_generator(val_generator_f2, x_data.shape[0])
    thresholded_pred = (np.array(raw_pred) > classifier_threshold).astype(int)
    l_f2_score = fbeta_score(y_data, thresholded_pred, beta=2, average=score_averaging_method)
    return l_f2_score

class F2_Validation(k.callbacks.Callback):
    def __init__(self, x_data, y_data):
        super(F2_Validation, self).__init__()
        # Ran into MemoryError when training DAGG_2 with 4 channels at epoch 50.
        # To try to get reduce memory usage, limit the number of samples to an arbitrary small number
        validation_num_samples = min(1280, x_data.shape[0])
        self.x_data = x_data[:validation_num_samples]
        self.y_data = y_data[:validation_num_samples]
    
    def on_train_begin(self, logs={}):
        self.f2_measures = []
    def on_epoch_end(self, epoch, logs={}):
        self.f2_measures.append(compute_f2_measure(self.model, self.x_data, self.y_data))

In [36]:
# Record performance metrics at the end of each epoch
class PerformanceHistory(k.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.train_losses = []
        self.val_losses = []
        self.train_accuracy = []
        self.val_accuracy = []

    def on_epoch_end(self, epoch, logs={}):
        self.train_losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.train_accuracy.append(logs.get('acc'))
        self.val_accuracy.append(logs.get('val_acc'))
        
perf_history = PerformanceHistory()

In [37]:
# early stopping prevents overfitting on training data
early_stop = EarlyStopping(monitor='val_loss',patience=3, min_delta=0, verbose=0, mode='auto')  # TODO tune min_delta?

# save only the best model, not the latest epoch model.
checkpoint = ModelCheckpoint(model_filepath, monitor='val_loss', verbose=1, save_best_only=True)

In [38]:
training_start_time = datetime.now()

history = {}
f2_history = []
#epochs_arr = [100, 100, 100]
#learn_rates = [0.001, 0.0001, 0.00001]

custom_gen = CustomImgGenerator()

adam = optimizers.Adam()

# https://github.com/fchollet/keras/issues/369
# https://github.com/fchollet/keras/blob/master/keras/losses.py
model.compile(loss='binary_crossentropy',
              optimizer=adam,
              metrics=['accuracy', 'recall', 'precision'])

for learn_rate, epochs in zip(learning_rate_schedule, max_epoch_per_learning_rate):
    print('learning rate :{}'.format(learn_rate))
    model.optimizer.lr.set_value(learn_rate) # https://github.com/fchollet/keras/issues/888
    # Remember to scale down the x values from 0-255 to 0-1
    #     tmp_history = model.fit(x_train / float(255),
    #                         y_train,
    #                         batch_size=batch_size,
    #                         nb_epoch=epochs,
    #                         verbose=1,
    #                         validation_data=(x_valid / float(255), y_valid),
    #                         callbacks=[f2_score_val, early_stop, checkpoint])

    # use our custom generators so we can perform augmentation
    # TODO split x_train into smaller batches for larger models
    train_gen = custom_gen.trainGen(x_train, y_train, batch_size)
    valid_gen = custom_gen.validationGen(x_valid, y_valid, batch_size)
    
    f2_score_val = F2_Validation(x_valid, y_valid)
    
    tmp_history = model.fit_generator(train_gen,
                        samples_per_epoch=num_samples_per_epoch,
                        nb_epoch=epochs,
                        validation_data=valid_gen,
                        nb_val_samples=number_validations,              
                        verbose=1,
                        callbacks=[f2_score_val, early_stop, checkpoint])
    
    for k, v in tmp_history.history.iteritems():
        history.setdefault(k, []).extend(v)
    print("f2 validation scores", f2_score_val.f2_measures)
    f2_history.extend(f2_score_val.f2_measures)


time_spent_trianing = datetime.now() - training_start_time
print('model training complete')

learning rate :0.001
Epoch 1/3
32383/32383 [==============================] - 14s - loss: 0.2226 - acc: 0.9145 - recall: 0.6101 - precision: 0.8446 - val_loss: 0.2051 - val_acc: 0.9180 - val_recall: 0.5937 - val_precision: 0.8815
Epoch 2/3
32383/32383 [==============================] - 13s - loss: 0.2067 - acc: 0.9186 - recall: 0.6353 - precision: 0.8454 - val_loss: 0.1973 - val_acc: 0.9221 - val_recall: 0.6285 - val_precision: 0.8744
Epoch 3/3
32383/32383 [==============================] - 13s - loss: 0.1976 - acc: 0.9223 - recall: 0.6586 - precision: 0.8485 - val_loss: 0.1927 - val_acc: 0.9220 - val_recall: 0.6221 - val_precision: 0.8806
('f2 validation scores', [0.79430446400027299, 0.80727153957717623, 0.81626665427174172])
learning rate :0.0001
Epoch 1/1
32

In [ ]:
print(model.summary())

In [ ]:
# model = load_model(data_dir + 'models/aggregate_model_20170517-062305.h5')
print(y_valid.shape)
print(y_valid.ndim)

In [ ]:
# use the validation data to compute some stats which tell us how the model is performing on the validation data set.
val_generator_score_board = validation_datagen.flow(
    x_valid,
    y_valid,
    batch_size=batch_size,
    shuffle=False)
p_valid = model.predict_generator(val_generator_score_board, number_validations)

In [ ]:
optimized_thresholds = f2_optimized_thresholds(y_valid, np.array(p_valid))

y_predictions = (np.array(p_valid) > optimized_thresholds).astype(int)

precision_s = precision_score(y_valid, y_predictions, average=score_averaging_method)
print('>>>> Overall precision score over validation set ' , precision_s)

recall_s = recall_score(y_valid, y_predictions, average=score_averaging_method)
print('>>>> Overall recall score over validation set ' , recall_s)

# F2 score, which gives twice the weight to recall
# 'samples' is what the evaluation criteria is for the contest
f2_score = fbeta_score(y_valid, y_predictions, beta=2, average=score_averaging_method)
print('>>>> Overall F2 score over validation set ' , f2_score)

In [ ]:
threshold_df = pd.DataFrame({'label':labels, 
                             'optimized_threshold':optimized_thresholds})
print(threshold_df)

In [ ]:
precision_l, recall_l, f2_score_l = calculate_stats_for_prediction(y_valid, y_predictions)

prediction_stats_df = pd.DataFrame({
    'label': labels, 
    'true_sum': np.sum(y_valid, axis=0),
    'predict_sum': np.sum(y_predictions, axis=0),
    'f2': f2_score_l,
    'recall': recall_l,
    'precision': precision_l
})

# reordering the columns for easier reading
prediction_stats_df = prediction_stats_df[['label', 'f2', 'recall', 'precision', 'true_sum', 'predict_sum']]
print(prediction_stats_df)

In [ ]:
print(history['val_acc'])

In [ ]:

filtered_data_mask_label = data_mask_label[data_mask]

data_set_name = os.path.basename(get_training_set_file_path(rescaled_dim))

record_model_scores(model_filename, 
                    model_id, 
                    history, 
                    f2_score, 
                    time_spent_trianing, 
                    num_channels,
                    config_file,
                    np.array_str(filtered_data_mask_label),
                    data_set_name)

In [ ]:
figures_dir = 'figures/' + model_id
makedirs(figures_dir)

# list all data in history
print('training history stats:')
print(history.keys())

# summarize history for f2 score
fig = plt.figure(figsize=(15, 10))
subplot0 = fig.add_subplot(231)
if hasattr(f2_history, 'f2_measures'):
    subplot0.plot(f2_history.f2_measures)
subplot0.set_title('f2 score')
subplot0.set_ylabel('f2 score')
subplot0.set_xlabel('epoch')
subplot0.legend(['val'], loc='upper left')

# summarize history for recall
subplot3 = fig.add_subplot(232)
subplot3.plot(history['recall'])
subplot3.plot(history['val_recall'])
subplot3.set_title('recall')
subplot3.set_ylabel('recall')
subplot3.set_xlabel('epoch')
subplot3.legend(['train', 'val'], loc='upper left')

# summarize history for precision
subplot2 = fig.add_subplot(233)
subplot2.plot(history['precision'])
subplot2.plot(history['val_precision'])
subplot2.set_title('precision')
subplot2.set_ylabel('precision')
subplot2.set_xlabel('epoch')
subplot2.legend(['train', 'val'], loc='upper left')

# summarize history for accuracy
subplot1 = fig.add_subplot(234)
subplot1.plot(history['acc'])
subplot1.plot(history['val_acc'])
subplot1.set_title('accuracy')
subplot1.set_ylabel('accuracy')
subplot1.set_xlabel('epoch')
subplot1.legend(['train', 'val'], loc='upper left')

# summarize history for loss
subplot4 = fig.add_subplot(235)
subplot4.plot(history['loss'])
subplot4.plot(history['val_loss'])
subplot4.set_title('model loss')
subplot4.set_ylabel('loss')
subplot4.set_xlabel('epoch')
subplot4.legend(['train', 'val'], loc='upper left')

# precision and recall for each label
subplot5 = fig.add_subplot(236)
colors = cm.rainbow(np.linspace(0, 1, len(prediction_stats_df['label'])))
subplot5.scatter(prediction_stats_df['precision'], prediction_stats_df['recall'], c=colors)
subplot5.set_title('precision & recall')
subplot5.set_xlabel('precision')
subplot5.set_ylabel('recall')
for i, txt in enumerate(prediction_stats_df['label']):
    subplot5.annotate(txt, (prediction_stats_df['precision'][i], prediction_stats_df['recall'][i]))

fig.savefig(figures_dir + '/stats_' + timestr + '.png')
#plt.show()

In [ ]:
# load pre-trained model
#model = load_model(data_dir + 'models/aggregate_model_20170521-141533.h5')
#print(model.summary())

# copied manually from stout
#recorded_thresholds = [0.13, 0.13, 0.1, 0.14, 0.05, 0.23, 0.17, 0.15, 0.18, 0.23, 0.09, 0.21, 0.17, 0.10, 0.20, 0.23, 0.1]

#image_generator = get_img_generator()




In [ ]:
# this is the configuration we will use for testing:
testset_datagen = image_generator.getTestGenenerator()

In [ ]:
if not is_test_set_in_cache(rescaled_dim):
    # populate the test dataset cache
    df_test = pd.read_csv(sample_submission_filepath)
    load_test_set(df_test, rescaled_dim)

real_submission_filepath = data_dir + 'my_submissions/submission_' + timestr + '.csv'
#prediction_filepath = data_dir + 'predictions/prediction_' + timestr + '.csv'

make_submission(model,
                optimized_thresholds,
                data_mask,
                testset_datagen, 
                rescaled_dim, 
                labels, 
                sample_submission_filepath,
                real_submission_filepath,
                need_norm_stats)

In [ ]:
total_exec_time = datetime.now() - start_time
print ('time spent to complete execution: {}'.format(total_exec_time))